In [ ]:
import os, pandas as pd, numpy as np, time
from dotenv import load_dotenv
from neo4j import GraphDatabase

from neo4j_graphrag.retrievers import VectorRetriever, VectorCypherRetriever, Text2CypherRetriever
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings import OpenAIEmbeddings

load_dotenv()
uri = os.getenv("NEO4J_URI")
user = os.getenv("NEO4J_USERNAME")
pwd  = os.getenv("NEO4J_PASSWORD")
openai_key = os.getenv("OPENAI_API_KEY")
index_name = os.getenv("VECTOR_INDEX_NAME", "chunkEmbeddings")

driver = GraphDatabase.driver(uri, auth=(user, pwd))
embedder = OpenAIEmbeddings(api_key=openai_key)
llm = OpenAILLM(model_name=os.getenv("OPENAI_MODEL","gpt-4o"), api_key=openai_key)

vector = VectorRetriever(driver=driver, index_name=index_name, embedder=embedder)
hybrid = VectorCypherRetriever(driver=driver, index_name=index_name, retrieval_query=None, embedder=embedder)
t2c    = Text2CypherRetriever(driver=driver, llm=llm, neo4j_schema=None)


In [ ]:
tests = pd.DataFrame([
    {"id":1, "question":"What are the main cybersecurity threats in financial services?", "expected_tool":"vector"},
    {"id":2, "question":"Which companies face regulatory challenges?", "expected_tool":"hybrid"},
    {"id":3, "question":"How many companies mention cloud computing?", "expected_tool":"t2c"},
    {"id":4, "question":"What products does Apple mention in their filings?", "expected_tool":"hybrid"},
    {"id":5, "question":"Count the risk factors for Microsoft", "expected_tool":"t2c"},
])
tests


In [ ]:
def run_all(q, k=5):
    t0=time.time(); v = vector.retrieve(q, top_k=k); t1=time.time()
    h = hybrid.retrieve(q, top_k=k); t2=time.time()
    try:
        c = t2c.retrieve(q); 
    except Exception as e:
        c = {"error": str(e)}
    t3=time.time()
    return {
        "vector_hits": len(v or []),
        "hybrid_hits": len(h or []),
        "t2c_ok": bool(c) and not c.get("error"),
        "lat_vector_ms": round((t1-t0)*1000,1),
        "lat_hybrid_ms": round((t2-t1)*1000,1),
        "lat_t2c_ms": round((t3-t2)*1000,1),
        "vector_sample": (v or [{}])[0],
        "hybrid_sample": (h or [{}])[0],
        "t2c_sample": c,
    }

rows = []
for _, r in tests.iterrows():
    res = run_all(r["question"])
    res["id"]=r["id"]; res["question"]=r["question"]; res["expected_tool"]=r["expected_tool"]
    # simple router guess
    ql = r["question"].lower()
    router_guess = "t2c" if any(w in ql for w in ["count","how many","list","top "]) else ("hybrid" if any(w in ql for w in ["apple","microsoft","google","aws","azure"]) else "vector")
    res["router_guess"]=router_guess
    rows.append(res)

eval_df = pd.DataFrame(rows)
eval_df[["id","question","expected_tool","router_guess","vector_hits","hybrid_hits","t2c_ok","lat_vector_ms","lat_hybrid_ms","lat_t2c_ms"]]


In [ ]:
def flag(row):
    return "✅" if row["router_guess"]==row["expected_tool"] else "⚠️"
eval_df.assign(match=eval_df.apply(flag, axis=1))[["id","question","expected_tool","router_guess","match",
                                                  "vector_hits","hybrid_hits","t2c_ok"]]
